In [1]:
import pandas as pd 
import numpy as np  
Question = "สัญญาณไฟจราจรสีส้มต้องทำอย่างไร" #input & คำถามที่จะทำการถาม 
corpus_df= pd.read_csv("data/corpus_df.csv")
corpus = corpus_df['corpus']

In [2]:
from FlagEmbedding import BGEM3FlagModel
model = BGEM3FlagModel('BAAI/bge-m3',use_fp16=True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

### 1st retrieve

In [3]:
import numpy as np
from tqdm import tqdm

ALL_final_ans_list_ALL = []
ALL_final_ans_list_Sparse_Dense = []
ALL_final_ans_list_sparse = []
ALL_final_ans_list_score_colbert = []
ALL_final_ans_list_Dense = []
 
batch_size = 10 

questions = Question
sentence_pairs = [[questions, j] for j in corpus]

listofscore = []
compute_Score = range(0, len(sentence_pairs), batch_size)

for i in compute_Score:
    batch_pairs = sentence_pairs[i:i+batch_size]
    allscore = model.compute_score(batch_pairs,
                                    max_passage_length=512,
                                    weights_for_different_modes=[0.4, 0.2, 0.4]) # sum: w[0]*dense_score + w[1]*sparse_score + w[2]*colbert_score
    listofscore.append(allscore) 

score_ALL = []
score_sparse_dense = []
score_colbert = []
score_dense = []
score_sparse = []

for score_dict in listofscore:
    score_dense.extend(score_dict['dense'])
    score_sparse_dense.extend(score_dict['sparse+dense'])
    score_colbert.extend(score_dict['colbert'])
    score_sparse.extend(score_dict['sparse'])
    score_ALL.extend(score_dict['colbert+sparse+dense'])

ALL_final_ans_list_ALL.append(score_ALL)
ALL_final_ans_list_Sparse_Dense.append(score_sparse_dense)
ALL_final_ans_list_sparse.append(score_sparse)
ALL_final_ans_list_score_colbert.append(score_colbert)
ALL_final_ans_list_Dense.append(score_dense)


In [4]:
topkindex = 15
topk15indices_ALL = np.argsort(np.array(ALL_final_ans_list_ALL), axis=1)[:, -topkindex:]

In [5]:
for indices in topk15indices_ALL:
    containptopk = []
    for i, index in enumerate(indices):
        if index < len(corpus_df):
            containptopk.append(' '.join(map(str, corpus_df.iloc[index].tolist())))  # Convert list to string and join elements

### Rerank

In [6]:
from FlagEmbedding import FlagReranker
rerankerbase = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True)

Rerank_base_passage= []
print(Question)
Input_text_for_rerank = containptopk
scores_base = []

for passage in Input_text_for_rerank:
    passage = str(passage)
    score_base = rerankerbase.compute_score([Question, passage], normalize=True)
    scores_base.append(score_base)
    print(passage[:20])

highest_scoring_index = scores_base.index(max(scores_base))

print(f"Highest: {max(scores_base)} - '{Input_text_for_rerank[highest_scoring_index]}'")
Rerank_base_passage.append(Input_text_for_rerank[highest_scoring_index])

print(Rerank_base_passage)

สัญญาณไฟจราจรสีส้มต้องทำอย่างไร
มาตรา 29 - ห้ามมิให้
มาตรา 25 - ผู้ขับขี่
มาตรา 24 - ผู้ขับขี่
มาตรา 15 - รถที่บรรท
มาตรา 76 - เมื่อคนเด
มาตรา 37 - การให้สัญ
มาตรา 21 - ผู้ขับขี่
มาตรา 50 -   ถ้ามีรถ
มาตรา 73 - ในกรณีที่
มาตรา 36 - ผู้ขับขี่
มาตรา 105 - คนเดินเท
มาตรา 23 - ผู้ขับขี่
มาตรา 106 - คนเดินเท
มาตรา 38 - การให้ไฟส
มาตรา 22 - ผู้ขับขี่
Highest: 0.1437877942733285 - 'มาตรา 22 - ผู้ขับขี่ต้องปฏิบัติตามสัญญาณจราจรหรือเครื่องหมายจราจรที่ปรากฏข้างหน้าในกรณีต่อไปนี้(1) สัญญาณจราจรไฟสีเหลืองอำพัน ให้ผู้ขับขี่เตรียมหยุดรถหลังเส้นให้รถหยุดเพื่อเตรียมปฏิบัติตามสัญญาณที่จะปรากฏต่อไปดังกล่าวใน (2) เว้นแต่ผู้ขับขี่ที่ได้เลยเส้นให้รถหยุดไปแล้วให้เลยไปได้(2) สัญญาณจราจรไฟสีแดงหรือเครื่องหมายจราจรสีแดงที่มีคำว่าหยุด ให้ผู้ขับขี่หยุดรถหลังเส้นให้รถหยุด(3) สัญญาณจราจรไฟสีเขียวหรือเครื่องหมายจราจรสีเขียวที่มีคำว่า ไป ให้ผู้ขับขี่ขับรถต่อไปได้ เว้นแต่จะมีเครื่องหมายจราจรกำหนดไว้เป็นอย่างอื่น(4)(1) สัญญาณจราจรไฟลูกศรสีเขียวชี้ให้เลี้ยวหรือชี้ให้ตรงไป หรือสัญญาณจราจรไฟสีแดงแสดงพร้อมกับสัญญาณจราจรไฟ

In [7]:
Rerank_base_passage

['มาตรา 22 - ผู้ขับขี่ต้องปฏิบัติตามสัญญาณจราจรหรือเครื่องหมายจราจรที่ปรากฏข้างหน้าในกรณีต่อไปนี้(1) สัญญาณจราจรไฟสีเหลืองอำพัน ให้ผู้ขับขี่เตรียมหยุดรถหลังเส้นให้รถหยุดเพื่อเตรียมปฏิบัติตามสัญญาณที่จะปรากฏต่อไปดังกล่าวใน (2) เว้นแต่ผู้ขับขี่ที่ได้เลยเส้นให้รถหยุดไปแล้วให้เลยไปได้(2) สัญญาณจราจรไฟสีแดงหรือเครื่องหมายจราจรสีแดงที่มีคำว่าหยุด ให้ผู้ขับขี่หยุดรถหลังเส้นให้รถหยุด(3) สัญญาณจราจรไฟสีเขียวหรือเครื่องหมายจราจรสีเขียวที่มีคำว่า ไป ให้ผู้ขับขี่ขับรถต่อไปได้ เว้นแต่จะมีเครื่องหมายจราจรกำหนดไว้เป็นอย่างอื่น(4)(1) สัญญาณจราจรไฟลูกศรสีเขียวชี้ให้เลี้ยวหรือชี้ให้ตรงไป หรือสัญญาณจราจรไฟสีแดงแสดงพร้อมกับสัญญาณจราจรไฟลูกศรสีเขียวชี้ให้เลี้ยวหรือชี้ให้ตรงไป ให้ผู้ขับขี่เลี้ยวรถหรือขับรถตรงไปได้ตามทิศทางที่ลูกศรชี้และต้องขับรถด้วยความระมัดระวัง และต้องให้สิทธิแก่คนเดินเท้าในทางข้ามหรือรถที่มาทางขวาก่อน(5) สัญญาณจราจรไฟกระพริบสีแดง ถ้าติดตั้งอยู่ที่ทางร่วมทางแยกใดเปิดทางด้านใดให้ผู้ขับขี่ที่มาทางด้านนั้นหยุดรถหลังเส้นให้รถหยุด เมื่อเห็นว่าปลอดภัยและไม่เป็นการกีดขวางการจราจรแล้ว จึงให้ขับรถ

In [18]:
!pip install langchain 

  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
     ---------------------------------------- 0.0/50.9 kB ? eta -:--:--
     ---------------------------------------- 50.9/50.9 kB 1.3 MB/s eta 0:00:00
  Using cached jsonpointer-2.4-py2.py3-none-any.whl.metadata (2.5 kB)
   ---------------------------------------- 0.0/974.0 kB ? eta -:--:--
   ------------------- ------------------- 481.3/974.0 kB 15.2 MB/s eta 0:00:01
   --------------------------------------- 974.0/974.0 kB 15.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/314.7 kB ? eta -:--:--
   --------------------------------------- 314.7/314.7 kB 19.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/124.9 kB ? eta -:--:--
   ---------------------------------------- 124.9/124.9 kB 7.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ----------------- ---------------------

In [8]:
import getpass
import os
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains import LLMChain

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")

In [21]:
pip install --upgrade --quiet  langchain-google-genai pillow

Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro")

In [10]:
template = """
คุณเป็นผู้เชี่ยวชาญด้านกฎหมายจราจร มีหน้าที่ในการนำข้อความทางกฎหมายเกี่ยวกับการละเมิดกฎจราจรเเละข้อปฎิบัติต่างๆมาตอบคำถามว่าคำถามที่ถามมานั้นผิดหรือไม่หรือจะต้องทำอย่างไรเเละอธิบายเพิ่มเติม ให้รายละเอียดและคำอธิบายเพิ่มเติมเพื่อให้ผู้ที่ไม่ใช่ผู้เชี่ยวชาญด้านกฎหมายเข้าใจได้ง่ายขึ้น

นี้คือคำถาม : {question}
ข้อความทางกฎหมาย: {section}

คำอธิบายโดยละเอียด:
"""

prompt = PromptTemplate(
    input_variables=["section","question"],
    template=template
)
llm_chain = LLMChain(prompt=prompt, llm=llm)


c:\Users\PC\anaconda3\envs\gpu_02\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [11]:
section = Rerank_base_passage
question = Question
inputs = {"section": section, "question": question}
response = llm_chain.run(inputs)
print(response)

c:\Users\PC\anaconda3\envs\gpu_02\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


**คำถาม:** สัญญาณไฟจราจรสีส้มต้องทำอย่างไร

**คำตอบ:**

ผู้ขับขี่ต้องเตรียมหยุดรถหลังเส้นให้รถหยุดเพื่อเตรียมปฏิบัติตามสัญญาณที่จะปรากฏต่อไป

**ข้อความทางกฎหมายที่เกี่ยวข้อง:**

มาตรา 22 (1) แห่งพระราชบัญญัติจราจรทางบก พ.ศ. 2522

**คำอธิบายเพิ่มเติม:**

เมื่อเห็นสัญญาณไฟจราจรสีส้ม ผู้ขับขี่ควรลดความเร็วและเตรียมหยุดรถหลังเส้นหยุดรถ โดยปกติแล้วสัญญาณไฟจราจรสีส้มจะปรากฏก่อนสัญญาณไฟจราจรสีแดง ดังนั้นจึงเป็นการแจ้งเตือนให้ผู้ขับขี่ชะลอรถและเตรียมตัวหยุดรถเมื่อสัญญาณไฟจราจรเปลี่ยนเป็นสีแดง

อย่างไรก็ตาม หากผู้ขับขี่ได้เลยเส้นหยุดรถไปแล้วก่อนที่สัญญาณไฟจราจรจะเปลี่ยนเป็นสีแดง ผู้ขับขี่สามารถขับรถต่อไปได้

นอกจากนี้ ผู้ขับขี่ควรระมัดระวังขณะขับผ่านสัญญาณไฟจราจรสีส้ม โดยเฉพาะอย่างยิ่งเมื่อมีคนเดินเท้าหรือรถคันอื่นมาทางขวา
